<a href="https://colab.research.google.com/github/sdegran2/python-learning-journal/blob/main/PokemonTeamBattle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Group 7: Ariana Lalehparvar, Fei Zheng, Steven DeGrange

Game Logic Update:

Battle Code Update: Integrate item usage within the battle sequence. Allow the player to pick one of three options to do on their turn:
Battle by picking a move to use on an opponent.
Use a Potion to heal their Pokémon during their turn.
Throw a Pokeball to attempt capturing the opponent’s Pokémon.
Team-Based Battle Logic: Allow the player to switch between different Pokémon on their team during battle if their active Pokémon faints.
Battle ends not when one Pokemon has fainted, but when an entire team has no Pokemon with HP above 0
Implement Exceptions:
Use try-except blocks to handle cases like:
Using a Potion on a fainted Pokémon.
Attempting to catch an opponent’s Pokémon when it doesn’t meet the capture conditions.
Attempting to switch to a Pokémon that’s already fainted.
"""

import random

# Define the Pokemon class
class Pokemon:
    # Class-level constants for Pokemon types and type effectiveness mappings
    pokeTypes = ("water", "fire", "grass")  # Basic trio of types for the game
    typeMaps = {  # Dictionary representing type effectiveness
        "fire": ["grass", "water"],  # Fire is strong against grass, weak against water
        "water": ["fire", "grass"],  # Water is strong against fire, weak against grass
        "grass": ["water", "fire"]   # Grass is strong against water, weak against fire
    }

    # Initialize Pokemon attributes
    def __init__(self, name, type, health, defense, moves):
        self.name = name         # Name of the Pokemon
        self.type = type         # Type of the Pokemon (water, fire, grass)
        self.health = health     # Current health of the Pokemon
        self.defense = defense   # Defense points of the Pokemon
        self.moves = moves       # List of Move objects (up to 4 moves per Pokemon)

    # Overwrite the __str__ method to return detailed info about the Pokemon object
    def __str__(self):
        # Display core stats and move list
        pokeStr = f"name={self.name}, type={self.type}, health={self.health}, defense={self.defense}, moves={len(self.moves)}"
        # Append each Move's details by iterating through self.moves
        if len(self.moves) > 0:
            pokeStr += f"\n\tAllowed moves for {self.name}:"
            for move in self.moves:
                pokeStr += f"\n\t\t{move}"  # Uses Move's __str__ method to show details
        return pokeStr  # Returns all collected information as a string

    # Return the type effectiveness mappings to check type interactions
    def getTypeMaps(self):
        return self.typeMaps

    # Simulate a battle between this Pokemon and another using a chosen move
    def battle(self, other, move):
        #print(f"{self.name} battles {other.name} with {move.name}")
        randomMove = random.choice(other.moves)  # Random move selected by opponent Pokemon
        print(f"{other.name} uses {randomMove.name}")
        maps = self.getTypeMaps()  # Access type effectiveness mappings

        # Check effectiveness of moves based on types and apply corresponding damage
        if maps[move.type][0] == randomMove.type:  # Super effective case
            print(f"{other.name} is weak to {self.name}")
            other.health -= self.getDamageNumber(move)  # Deal full damage to opponent
        elif maps[move.type][1] == randomMove.type:  # Resistant case
            print(f"{other.name} is strong to {self.name}")
            self.health -= other.getDamageNumber(randomMove) * 2  # Take double damage from opponent
        else:  # Neutral effectiveness case
            print(f"{self.name} is stronger in {self.type}")
            self.health -= other.getDamageNumber(move) / 2  # Deal half damage

        # Display updated health after the battle round
        print(f"{self.name} health: {self.health}")
        print(f"{other.name} health: {other.health}")

    # Calculate damage for a move by adding random damage variation
    def getDamageNumber(self, move):
        randomDamageNumber = random.randint(1, 3) + move.damage  # Base damage + random component
        print(f"{move.name} gives {randomDamageNumber} damage")
        return randomDamageNumber  # Return calculated damage value

# Define the Move class, representing individual moves that a Pokemon can perform
class Move:
    def __init__(self, name, damage, type):
        self.name = name       # Name of the move (e.g., "acid")
        self.damage = damage   # Damage value associated with the move
        self.type = type       # Type of the move (water, fire, grass)

    # Overwrite __str__ to display move information in a structured format
    def __str__(self):
        return f"name={self.name}, type={self.type}, damage={self.damage}"

    # Getter for move damage (for potential further customization)
    def getDamage(self):
        return self.damage

class Item:
    def __init__(self, name, quantity):
        self.name = name
        self.quantity = quantity

    def __str__(self):
        return f"name={self.name}, quantity={self.quantity}"

class Potion(Item): #potion is used for healing fix health
    def __init__(self, name, quantity, heal_amount):
        super().__init__(name, quantity)
        self.heal_amount = heal_amount

    def use(self, pokemon):
        if self.quantity > 0:
            pokemon.health += self.heal_amount
            print(f"Healing {pokemon.name}")
            print(f"{pokemon.name} health: {pokemon.health}")
            self.quantity -= 1
        else:
            print(f"No {self.name} left")

class Pokeball(Item): #pokeball can capture weak pokemon
    def __init__(self, name, quantity, catch_rate):
        super().__init__(name, quantity)
        self.catch_rate = catch_rate

    def use(self, pokemon):
        if self.quantity > 0:
            if random.random() < self.catch_rate:
                print(f"{pokemon.name} has been caught!")
                self.quantity -= 1
            else:
                print(f"{pokemon.name} has failed to catch!")
        else:
            print(f"No {self.name} left")

class Team:
    def __init__(self, name, pokemon_list):
        self.name = name
        self.pokemon_list = pokemon_list
    def __str__(self):
        return f"name={self.name}, pokemon_list={self.pokemon_list}"
    def add_pokemon(self, pokemon):
        self.pokemon_list.append(pokemon)

    def remove_pokemon(self, pokemon):
        self.pokemon_list.remove(pokemon)

    def isAllFainted(self):
        for pokemon in self.pokemon_list:
            if pokemon.health > 0:
                return False
        return True

    def getStronestPokemon(self):
        strongest_pokemon = self.pokemon_list[0]
        for pokemon in self.pokemon_list:
            if pokemon.health > strongest_pokemon.health:
                strongest_pokemon = pokemon
        return strongest_pokemon

    def getMoves(self):
        return self.pokemon_list[0].moves

    def getWeakestPokemon(self):
        weakest_pokemon = self.pokemon_list[0]
        for pokemon in self.pokemon_list:
            if pokemon.health < weakest_pokemon.health:
                weakest_pokemon = pokemon
        return weakest_pokemon

    def heal(self, potion):
        for pokemon in self.pokemon_list:
            if pokemon.health <= 0:
                potion.use(pokemon)

    def capture(self, other, pokeball):
        weakest_pokemon = self.getWeakestPokemon()
        if weakest_pokemon.health <= 0:
            pokeball.use(weakest_pokemon)
            self.remove_pokemon(weakest_pokemon)
            other.add_pokemon(weakest_pokemon)
            print(f"{weakest_pokemon.name} has been captured!")
        else:
            print(f"{weakest_pokemon.name} is still alive!")


    def battle(self, other, move):
        if not self.isAllFainted() and not other.isAllFainted():
            strongest_pokemon = self.getStronestPokemon()
            other_pokemon = other.getStronestPokemon()
            while True:
                if strongest_pokemon.health <= 0:
                    strongest_pokemon = self.getStronestPokemon()
                    break
                if other_pokemon.health <= 0:
                    other_pokemon = other.getStronestPokemon()
                if strongest_pokemon.health <= 0 or other_pokemon.health <= 0:
                    break
                strongest_pokemon.battle(other_pokemon, move)
                print(f"{strongest_pokemon.name} battles {other_pokemon.name} with {move.name}")

        else:
            if self.isAllFainted():
                print(f"{other.name} has won the battle!")
            else:
                print(f"{self.name} has won the battle!")




# Create instances of Move objects to use in battles
acidMove = Move("acid", 3, "water")
auraSphere = Move("aura", 2, "grass")
aeroblast = Move("aeroblast", 2, "fire")
megaPunch = Move("megapunch", 5, "fire")

fireLizardPoke = Pokemon("FireLizardPoke", "fire", 10, 10, [acidMove, auraSphere, aeroblast, megaPunch])
waterBugPoke = Pokemon("WaterBugPoke", "water", 10, 10, [acidMove, auraSphere, aeroblast, megaPunch])
grassDragonPoke = Pokemon("GrassDragonPoke", "grass", 10, 10, [acidMove, auraSphere, aeroblast, megaPunch])
cpuTeam = Team("RobomonTeam", [])
cpuTeam.add_pokemon(fireLizardPoke)
cpuTeam.add_pokemon(waterBugPoke)
cpuTeam.add_pokemon(grassDragonPoke)


myTeam = Team("MeemonTeam", [])
fireballPokemon = Pokemon("FireballPokemon", "fire", 10, 10, [acidMove, auraSphere, aeroblast, megaPunch])
waterballPokemon = Pokemon("WaterballPokemon", "water", 10, 10, [acidMove, auraSphere, aeroblast, megaPunch])
grassBeanPokemon = Pokemon("GrassBeanPokemon", "grass", 10, 10, [acidMove, auraSphere, aeroblast, megaPunch])
myTeam.add_pokemon(fireballPokemon)
myTeam.add_pokemon(waterballPokemon)
myTeam.add_pokemon(grassBeanPokemon)

potion = Potion("potion", 1, 10)
pokeball = Pokeball("pokeball", 1, 0.5)

# Main game loop - runs until one Pokemon's health is depleted
while True:
    if cpuTeam.isAllFainted():  # Check if the CPU Pokemons have all fainted
        print(f"{cpuTeam.name} have all fainted. You win!")
        break
    elif myTeam.isAllFainted():  # Check if the player Pokemons have all fainted
        print(f"{myTeam.name} have all fainted. Game over.")
        break

    # Player chooses a move to use
    mode = input ("Please select from battle, heal or pokeball: ").lower()
    while mode not in ["battle", "heal", "pokeball"]:
        mode = input ("Please select from battle, heal or pokeball: ").lower()

    if mode == "battle":
        myMove = input(f"Please select a move for {myTeam.name} from acid, aura, aeroblast, megapunch: ").lower()
        while myMove not in [move.name for move in myTeam.getMoves()]:  # Validate chosen move
            myMove = input(f"Please select a move for {myTeam.name} from acid, aura, aeroblast, megapunch: ").lower()

        print(f"{myTeam.name} uses {myMove}")

        # Find and use the selected move in battle
        for move in myTeam.getMoves():
            if move.name == myMove:
                myTeam.battle(cpuTeam, move)  # Execute battle round with chosen move
    elif mode == "heal":
        myTeam.heal(potion)
    elif mode == "pokeball":
        myTeam.capture(cpuTeam, pokeball)


Please select from battle, heal or pokeball: battle
Please select a move for MeemonTeam from acid, aura, aeroblast, megapunch: aura
MeemonTeam uses aura
FireLizardPoke uses acid
FireLizardPoke is weak to FireballPokemon
aura gives 3 damage
FireballPokemon health: 10
FireLizardPoke health: 7
FireballPokemon battles FireLizardPoke with aura
FireLizardPoke uses aeroblast
FireLizardPoke is strong to FireballPokemon
aeroblast gives 3 damage
FireballPokemon health: 4
FireLizardPoke health: 7
FireballPokemon battles FireLizardPoke with aura
FireLizardPoke uses acid
FireLizardPoke is weak to FireballPokemon
aura gives 3 damage
FireballPokemon health: 4
FireLizardPoke health: 4
FireballPokemon battles FireLizardPoke with aura
FireLizardPoke uses megapunch
FireLizardPoke is strong to FireballPokemon
megapunch gives 7 damage
FireballPokemon health: -10
FireLizardPoke health: 4
FireballPokemon battles FireLizardPoke with aura
Please select from battle, heal or pokeball: heal
Healing FireballPokemo